In [1]:
import nltk
import re
import multiprocessing as mp
import plotly.express as px

Concordance

In [2]:
import nltk
from tqdm import tqdm
import os
import sys

def concordanceToFile(conc):
    new_file = '../txts/concordance/concordance_' + conc + '.txt'
    conc_save = text.concordance_list(conc, lines=1000000)
    
    with open(new_file, 'w', encoding='utf8') as fin:
        for i, line in enumerate(tqdm(conc_save, total=len(conc_save))):
                fin.write(str(line).strip("ConcordanceLine(") + "\n")

file_path = '../txts/all/russianNoRetweets.txt'
tokens = ""

with open(file_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(tqdm(f.readlines(), total=1000000)):
        tokens+=line
        if i == 1000000:
            tokens = nltk.word_tokenize(tokens)
            break

text = nltk.Text(tokens)

vocab = ["я", "мы", "ты", "oни", "нас", "вы", "тебя", "меня", "тебе", "вами", "мне"]

for i in vocab:
    concordanceToFile(i)

100%|██████████| 2207/2207 [00:00<00:00, 62015.66it/s]
0it [00:00, ?it/s]
100%|██████████| 2043/2043 [00:00<00:00, 52868.07it/s]


Processing Concordance

In [3]:
def writeToFile(filename, left_tokenize, right_tokenize, query):
    obj=[query],left_tokenize,right_tokenize

    with open('../txts/POS-tagged/' + filename, 'a', encoding='utf8') as fin:
        fin.write(str(obj) + "\n")

In [15]:
# Creates a new file, takes the base concordance file and runs POS tagging.
# This code strips the concordance down to left and right of the query (word concordance query)
def stripConcordance(query):
    stripped_string = "".lower()
    with open('../txts/concordance/concordance_'+ query + '.txt', 'r', encoding='utf8') as f:
        for line in f.readlines():
            
            # Strips the left and right side of the concordance of unwanted parts.
            left = line[line.find("left_print="):line.find("right_")].strip("left_print=")
            left_re = re.sub(r'[^[а-яА-Я0-9 ]+', '', left)
            right = line[line.find("right_print="):line.find("line=")].strip("right_print=")
            right_re = re.sub(r'[^[а-яА-Я0-9 ]+', '', right)
            
            # Combines the left and right side of the concordance into a single string.
            stripped_string += left_re
            stripped_string += " "
            stripped_string += right_re
            
            left_tokenize = nltk.pos_tag(nltk.word_tokenize(left_re))
            right_tokenize = nltk.pos_tag(nltk.word_tokenize(right_re))
            writeToFile('POS-concordance-'+ query +'.txt', left_tokenize, right_tokenize, query)
        
        return pos_tag_rus(stripped_string, query)

In [5]:
# Removes words used to obtain data as well as any words that have snuck into the concordance.
def posStripper(stripped_string):
    stripped_string = stripped_string.replace("conspiracy", "")
    stripped_string = stripped_string.replace("propaganda", "")
    stripped_string = stripped_string.replace("misinformation", "")
    stripped_string = stripped_string.replace("https", "")
    return stripped_string       

POS tagging processed concordance

In [206]:
import pymorphy2
# https://pymorphy2.readthedocs.io/en/stable/user/guide.html
def pos_tag_rus(stripped_string, query):
    morph = pymorphy2.MorphAnalyzer()
    token = nltk.word_tokenize(stripped_string)

    k = []
    for i in token:
        j = morph.parse(i)[0]
        k.append([i, j.tag.POS])
    nouns = get_nouns(k)
    verbs = get_verbs(k)
    return (verbs, nouns)

In [45]:
def get_verbs(ls):
    most_common_verbs = []
    for i in ls:
        if i[1] == "VERB":
            most_common_verbs.append(i[0])
    verb_frq = nltk.FreqDist(verb for verb in most_common_verbs)
    most_common_verbs = verb_frq.most_common(20)
    return most_common_verbs

In [48]:
# get all nouns and return the 20 most common
def get_nouns(ls):
    most_common_nouns = []
    for i in ls:
        if i[1] == 'NOUN':
            most_common_nouns.append(i[0])
    noun_frq = nltk.FreqDist(noun for noun in most_common_nouns)
    most_common_nouns = noun_frq.most_common(20)

    return most_common_nouns

In [ ]:
vocab = ["я", "мы", "ты", "oни", "нас", "вы", "тебя", "меня", "тебе", "вами", "мне"]

for i in vocab:
    print(i)
    print(stripConcordance(i))

In [572]:
# Python program to sort a list of tuples by the second Item
  
# Function to sort the list of tuples by its second item
def Sort_Tuple(tup): 
    # getting length of list of tuples
    lst = len(tup)
    for i in range(0, lst):
        for j in range(0, lst-i-1):
            if (tup[j][1] > tup[j + 1][1]): 
                temp = tup[j] 
                tup[j]= tup[j + 1] 
                tup[j + 1]= temp 
    return tup

def remove_single_letter_words(tup):
    for i in tup:

        if i[0] == 'е':

            tup.remove(i)
    return tup

[('распространяете', 39), ('будете', 41), ('может', 45), ('ходят', 50), ('понимаете', 50), ('хочешь', 54), ('хотите', 58), ('ненавижу', 60), ('думаете', 63), ('можете', 81), ('знаешь', 89), ('знаете', 94), ('можешь', 98)]


In [515]:
vocab = ["я", "мы", "ты", "oни", "нас", "вы", "тебя", "меня", "тебе", "вами", "мне"]

# Raw data
# The Russian language has different forms of pronouns which are very similar
# Therefore I have chosen to group together the pronouns which mean the same
# but are used in different settings (tenses, groups, male, female etc).
# I chose to append all of the data to a list.
temp_You=[]
temp_You.append(stripConcordance(vocab[2])[0])
temp_You.append(stripConcordance(vocab[5])[0])
temp_You.append(stripConcordance(vocab[6])[0])
temp_You.append(stripConcordance(vocab[8])[0])
temp_You.append(stripConcordance(vocab[9])[0])

verbs_You = []
for i in temp_You:
    for j in i:
        verbs_You.append(j)

temp_Me = []
temp_Me.append(stripConcordance(vocab[7])[0])
temp_Me.append(stripConcordance(vocab[10])[0])

verbs_Me=[]
for i in temp_Me:
    for j in i:
        verbs_Me.append(j)

verbs_Us = stripConcordance(vocab[4])[0]
verbs_We = stripConcordance(vocab[1])[0]
verbs_They = stripConcordance(vocab[3])[0]

In [595]:
print(type(verbs_Me))

<class 'list'>


In [576]:
# Sorts the tuple in order of frequency

verbs_You = Sort_Tuple(verbs_You)
verbs_Me = Sort_Tuple(verbs_Me)
verbs_Us = Sort_Tuple(verbs_Us)
verbs_We = Sort_Tuple(verbs_We)
verbs_They = Sort_Tuple(verbs_They)

verbs_You = remove_single_letter_words(verbs_You)
verbs_Me = remove_single_letter_words(verbs_Me)
verbs_Us = remove_single_letter_words(verbs_Us)
verbs_We = remove_single_letter_words(verbs_We)
verbs_They = remove_single_letter_words(verbs_They)

print(verbs_Me)

[('хватит', 17), ('были', 18), ('был', 18), ('стало', 21), ('нравятся', 22), ('сказали', 22), ('говорит', 23), ('буду', 24), ('может', 24), ('хватит', 25), ('сказал', 25), ('имеет', 26), ('был', 26), ('бесит', 28), ('были', 28), ('федай', 28), ('хочу', 29), ('хочу', 29), ('могу', 29), ('подписывайся', 30), ('оказал', 30), ('дошли', 30), ('буду', 31), ('хочется', 31), ('была', 32), ('знаю', 32), ('знаю', 33), ('могу', 34), ('оказывает', 35), ('ходят', 35), ('ходят', 37), ('может', 38), ('будет', 42), ('будет', 45), ('было', 60), ('было', 67), ('нравится', 119), ('кажется', 184)]


In [592]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

# Constructors
verbs = []
verbCounter=[]
verbCounterPercentage = []
pronouns=["me","you", "us"]

# Simple function to find a percentage.
def findPercentage(num, total):
    quotient = num / total
    return quotient*100

# returns total number of verbs associated with a pronoun.
def findTotalFreq(verbListTuple):
    total=[]
    for i,j in verbListTuple:
        total.append(j)
    return sum(total)

# Function to find the rounded down percent of a single verb in
# comparison to all other verbs associated with the pronoun,
def findPerecentTotalFreq(verbListTuple):
    total = findTotalFreq(verbListTuple)
    temp = []
    for i,j in verbListTuple:
        temp.append(math.floor(findPercentage(j, total)))
        #print(i, math.floor(findPercentage(j, total)))

    verbCounterPercentage.append(temp)

# Function to split a list of tuples into two seperate ordered lists.
# Here we add the verb to "verbs" and the frequency of the verb to "verbCounter"
def splitTuple(verbListTuples):
    temp = []
    tempCount = []
    for i, j in enumerate(verbListTuples):
        temp.append(j[0])
        tempCount.append(j[1])
        if i == 19:
            verbCounter.append(tempCount)
            verbs.append(temp)
            temp=[]
            tempCount=[]
            break

# Uses the ST function to seperate the verb and frequency count and add them to seperate lists.
splitTuple(verbs_Me)
splitTuple(verbs_You)
splitTuple(verbs_Us)

# Using the FPTF function to add each perecnt frequency to an ordered list.
findPerecentTotalFreq(verbs_Me)
findPerecentTotalFreq(verbs_You)
findPerecentTotalFreq(verbs_Us)

In [597]:
import plotly.express as px
import pandas as pd

# Function which creates a dataframe containing
# the verb tuple and associated pronoun. 
def dataframer(verb_tuple, pnoun):
    data = {}
    counter = []
    verbs = []

    for i in verb_tuple:
        j, k = i
        verbs.append(j)
        counter.append(k)

    for pronoun in range(len(verbs)):
        data["Pronoun"] = pnoun
        data["Verb"] = verbs
        data["Count"] = counter

    df_new = pd.DataFrame(data=data)
    return df_new

df = pd.DataFrame(columns=["Pronoun", "Verb", "Count"])

you_df = dataframer(verbs_You[-5:], "You")
me_df = dataframer(verbs_Me[-5:], "Me")
us_df = dataframer(verbs_Us[-5:], "Us")
we_df = dataframer(verbs_We[-5:], "We")
they_df = dataframer(verbs_They[-5:], "They")

new = df.append(you_df)
new = new.append(me_df)
new = new.append(us_df)
new = new.append(we_df)
new = new.append(they_df)

new = new.sort_values(by=['Count'], ascending=True)

fig = px.bar(new, x="Pronoun", y="Count", color="Pronoun", title="5 Most Common Verbs Found Alongside Russian Pronouns", text=new['Verb'], color_discrete_sequence=px.colors.sequential.Viridis)
fig.update_layout(hovermode="closest")
fig.show()

**Translated Table of Russian Verbs**



| Us             | We           | Me           | You          |
| :------------- | :----------: | -----------: | -----------: |
|  Walk          |  Know        |  Seems       |  (You) Can   |
|  To Be         |  Will        |  Like        |  Know        |
|  Will          |  Can         |  Was         |  Know (diff) |
|  Ruin          |  See         |  (repeat?    |  Can (You)   |
|  Can           |  Will        |  Will        |  Think       |



In [ ]:
#reference for nltk multi processing
#https://datascience.blog.wzb.eu/2017/06/19/speeding-up-nltk-with-parallel-processing/
corpus = {f_id: nltk.corpus.gutenberg.raw(f_id)
          for f_id in nltk.corpus.gutenberg.fileids()}

def tokenize_and_pos_tag(pair):
    f_id, doc = pair
    return f_id, nltk.pos_tag(nltk.word_tokenize(doc))

if __name__ == '__main__':
    # automatically uses mp.cpu_count() as number of workers
    # mp.cpu_count() is 4 -> use 4 jobs
    with mp.Pool() as pool:
        tokens = pool.map(tokenize_and_pos_tag, corpus.items())